In [76]:
using LinearAlgebra

# N spins
N = 14
Dim = 2^N

# couplings
J_X = -1
J_Y = -1
J_Z = -1
B = 0
#decay
alpha = 15

#function for the distance 1/(r_ij ^alpha)
function r_distance(index_i, index_j, alpha)
    dist = abs(index_i - index_j)^alpha
    return (1 / dist)
end

# initialise Hamiltonian
Hamiltonian = zeros(Float32, Dim, Dim)

for Ket = 0:Dim-1
    Diagonal = 0.0

    for i = 0:N-1 #we're doing i!=j so here looping first for i<j (effectively half a sum). Then we multiply by 2 every Hamiltonian contribution to account for all pairs. 
        for j = i+1:N-1

            #S^Z part, similar to before 
            Spin1 = 2 * ((Ket >> i) & 1) - 1
            Spin2 = 2 * ((Ket >> j) & 1) - 1
            Diagonal += -2.0 * r_distance(i, j, alpha) * J_Z * 0.25 * Spin1 * Spin2


            #S^X part 
            bit_i = 2^i
            bit_j = 2^j
            Bra = Ket ⊻ bit_i ⊻ bit_j
            Hamiltonian[Bra+1, Ket+1] += -2.0 * J_X * 0.25 * r_distance(i, j, alpha)

            si = (Ket >> i) & 1
            sj = (Ket >> j) & 1

            #S^Y part- not all terms have the same sign from the fact that S^Y=1/2i (S^+-S^-). I expanded S^Y*S^Y to check which terms would have an overall minus sign. 

            Bra = Ket ⊻ (bit_i) ⊻ (bit_j)
            sign = (si == sj) ? 1 : -1
            Hamiltonian[Bra+1, Ket+1] += -(-0.25) * 2 * J_Y * r_distance(i, j, alpha) * sign


        end
        Hamiltonian[Ket+1, Ket+1] = Diagonal
        for SpinIndex = 0:N-1
            bit = 2^SpinIndex   #The "label" of the bit to be flipped
            Bra = Ket ⊻ bit    #Binary XOR flips the bit
            Hamiltonian[Bra+1, Ket+1] = -0.5 * B
        end
    end
end




In [77]:
using LinearAlgebra
Diag = eigen(Hamiltonian)

GroundState = Diag.vectors[:, 1]  #this gives the groundstate eigenvector
energy = Diag.values

println("Groundstate eigenvector:$(GroundState)")
println("Corresponding eigenvalue: $(energy[1]/N)")


Groundstate eigenvector:Float32[1.2578599f-23, 9.9115494f-23, -1.4165226f-22, 3.606725f-24, 8.38998f-24, 6.820626f-23, -1.1257564f-22, 5.0264493f-24, -1.5783015f-22, -1.8796052f-22, 3.0134656f-22, -7.932214f-23, -2.146459f-22, -3.3947957f-23, -9.6297166f-23, -1.3242825f-22, 8.175598f-23, -7.1363f-23, -1.8660819f-23, 1.2611378f-22, 4.1211375f-23, 4.843522f-23, -1.04203856f-23, -5.333338f-23, 9.0768273f-23, 1.053284f-22, -1.9660143f-22, 7.2111176f-23, 7.0942817f-23, -3.9119923f-22, -8.649722f-23, -6.961728f-22, 3.8671325f-23, 8.496422f-23, 5.4584467f-23, 4.8039868f-23, 2.6382596f-23, -2.928629f-22, -1.7627756f-22, 2.8519967f-22, 3.81918f-25, 2.5477509f-22, 7.146713f-23, -3.000428f-23, 3.0758416f-22, 1.9786419f-22, -1.0180289f-22, 7.33191f-21, 4.89461f-23, -1.2503999f-23, 8.926655f-23, -4.5298943f-23, -4.0014554f-22, 9.6779324f-23, -8.7497776f-23, -3.274973f-20, 3.6164388f-23, -5.936515f-23, 1.0161397f-22, 7.877826f-20, -1.734256f-22, -1.0300181f-19, 5.0179548f-20, 2.3293406f-19, -1.16370

In [9]:
magnetizationX = 0.0
for Ket = 0:Dim-1  #Loop over Hilbert Space
    for Bra = 0:Dim-1  #The "easy" (slow) way to do this 
        SumSx = 0.0
        for SpinIndex = 0:N-1  #Loop over spin index 
            bit = 2^SpinIndex   #The bit to be flipped by S+ or S-
            Bra2 = Ket ⊻ bit    #Binary XOR flips the bit
            if Bra == Bra2
                SumSx += 0.5 #the factor of 1/2 above
            end
        end
        magnetizationX += SumSx * GroundState[Ket+1] * GroundState[Bra+1]
    end #Bra
end #Ket
println(magnetizationX / N)

0.5000000968575504
